In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
import os
from tqdm.notebook import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
from time import localtime, strptime, time

In [2]:
df = pd.read_csv("data.csv")

/tmp/ipykernel_3564/3477112522.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data.csv")


In [3]:
class ListDataset(Dataset):
    def __init__(self, original_list):
        self.original_list = original_list

    def __len__(self):
        return len(self.original_list)

    def __getitem__(self, i):
        return self.original_list[i]

dataset = ListDataset(df["text"].values.tolist())

In [4]:
df['V'].last_valid_index()

559000

In [5]:
def get_checkpoint(epoch=0, run="latest", folder_path="results"):
    if run == "latest":
        run = max(os.listdir(folder_path), key=lambda x: strptime(x, "%b-%d-%y-%H:%M:%S"))

    checkpoint = sorted(os.listdir(f"{folder_path}/{run}"), key=lambda x: int(x.split("-")[1]))[epoch-1]

    return f"{folder_path}/{run}/{checkpoint}"

c = get_checkpoint(3, run="Dec-30-23-23:32:39", folder_path="../M1/results")
model = AutoModelForSequenceClassification.from_pretrained(c)
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [6]:
class RegPipeline(TextClassificationPipeline):
    def postprocess(self, model_outputs):
        best_class = model_outputs["logits"]
        return torch.sigmoid(best_class) * 4 + 1

In [7]:
pipe = RegPipeline(model=model, tokenizer=tokenizer)

In [8]:
start = 559000
dataset = ListDataset(df["text"].values.tolist()[start:])

for i, line in tqdm(enumerate(pipe(dataset, batch_size=8, truncation=True, max_length=128)), total=len(dataset)):
    df.loc[i+start, list("VAD")] = line[0].numpy()
    if (start+i) % 1000 == 0: df.to_csv("data.csv", index=False)

  0%|          | 0/1769 [00:00<?, ?it/s]

In [9]:
df.to_csv("data.csv", index=False)